In [1]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
import random
import cv2
import os
import tensorflow as tf
from tqdm import tqdm

import tensorflow as tf
tf.test.gpu_device_name()
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Mounted at /content/drive


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14476184848920465612
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14343274496
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 8480291023511790585
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
good_frames = '/content/drive/MyDrive/MajorP/datax/sharp/PNEUMONIA'
blur_frames= '/content/drive/MyDrive/MajorP/datax/blur'

In [3]:
clean_frames = []
for file in tqdm(sorted(os.listdir(good_frames))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(good_frames + '/' + file, target_size=(224,224))
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    clean_frames.append(image)

clean_frames = np.array(clean_frames)
blurry_frames = []
for file in tqdm(sorted(os.listdir(blur_frames))):
  if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
    image = tf.keras.preprocessing.image.load_img(blur_frames + '/' + file, target_size=(224,224))
    image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
    blurry_frames.append(image)

blurry_frames = np.array(blurry_frames)



100%|██████████| 3875/3875 [01:16<00:00, 50.90it/s]


In [4]:
x = clean_frames;
y = blurry_frames;

In [5]:
from sklearn.model_selection import train_test_split

In [ ]:


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [3]:
# import os
# import numpy as np
# import tensorflow as tf
# from tqdm import tqdm
# from sklearn.model_selection import train_test_split

# def load_images(image_dir):
#     for file in tqdm(sorted(os.listdir(image_dir))):
#         if any(extension in file for extension in ['.jpg', 'jpeg', '.png']):
#             image = tf.keras.preprocessing.image.load_img(os.path.join(image_dir, file), target_size=(224,224))
#             image = tf.keras.preprocessing.image.img_to_array(image).astype('float32') / 255
#             yield image

# # Load clean frames using a generator function
# clean_gen = load_images(good_frames)
# clean_frames = np.array([next(clean_gen) for _ in range(len(os.listdir(good_frames)))])

# # Load blurry frames using a generator function
# blurry_gen = load_images(blur_frames)
# blurry_frames = np.array([next(blurry_gen) for _ in range(len(os.listdir(blur_frames)))])

# # Split the data into training and testing sets using generator functions
# batch_size = 32
# train_gen = ((x, y) for x, y in zip(clean_frames[:len(clean_frames)//5*4], blurry_frames[:len(blurry_frames)//5*4]))
# test_gen = ((x, y) for x, y in zip(clean_frames[len(clean_frames)//5*4:], blurry_frames[len(blurry_frames)//5*4:]))
# train_ds = tf.data.Dataset.from_generator(lambda: train_gen, output_types=(tf.float32, tf.float32))
# train_ds = train_ds.batch(batch_size)
# test_ds = tf.data.Dataset.from_generator(lambda: test_gen, output_types=(tf.float32, tf.float32))
# test_ds = test_ds.batch(batch_size)


 80%|████████  | 3111/3875 [00:41<00:10, 74.60it/s] 


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
import cv2
import os
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Load sharp and blur images
sharp_images = []
blur_images = []

for file in os.listdir('/content/drive/MyDrive/MajorP/datax/sharp/PNEUMONIA'):
    img = cv2.imread(os.path.join('/content/drive/MyDrive/MajorP/datax/sharp/PNEUMONIA', file))
    img = cv2.resize(img, (224, 224))
    img = img/255.0
    sharp_images.append(img)
    
for file in os.listdir('/content/drive/MyDrive/MajorP/datax/blur'):
    img = cv2.imread(os.path.join('/content/drive/MyDrive/MajorP/datax/blur', file))
    img = cv2.resize(img, (224, 224))
    img = img/255.0
    blur_images.append(img)

sharp_images = np.array(sharp_images)
blur_images = np.array(blur_images)

# Split into train and validation sets
train_sharp, val_sharp, train_blur, val_blur = train_test_split(sharp_images, blur_images, test_size=0.2, random_state=42)




In [ ]:
inputs = Input(shape=(224, 224, 3))

conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = Conv2D(512, 3, activation='relu', padding='same')(pool3)
conv4 = Conv2D(512, 3, activation='relu', padding='same')(conv4)
drop4 = Dropout(0.5)(conv4)
pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

conv5 = Conv2D(1024, 3, activation='relu', padding='same')(pool4)
conv5 = Conv2D(1024, 3, activation='relu', padding='same')(conv5)
drop5 = Dropout(0.5)(conv5)

up6 = Conv2D(512, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(drop5))
merge6 = concatenate([drop4, up6], axis=3)
conv6 = Conv2D(512, 3, activation='relu', padding='same')(merge6)
conv6 = Conv2D(512, 3, activation='relu', padding='same')(conv6)

up7 = Conv2D(256, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv6))
merge7 = concatenate([conv3, up7], axis=3)
conv7 = Conv2D(256, 3, activation='relu', padding='same')(merge7)
conv7 = Conv2D(256, 3, activation='relu', padding='same')(conv7)

up8 = Conv2D(128, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv7))
merge8 = concatenate([conv2, up8], axis=3)
conv8 = Conv2D(128, 3, activation='relu', padding='same')(merge8)
conv8 = Conv2D(128, 3, activation='relu', padding='same')(conv8)

up9 = Conv2D(64, 2, activation='relu', padding='same')(UpSampling2D(size=(2, 2))(conv8))
merge9 = concatenate([conv1, up9], axis=3)
conv9 = Conv2D(64, 3, activation='relu', padding='same')(merge9)
conv9 = Conv2D(64, 3, activation='relu', padding='same')(conv9)

conv10 = Conv2D(3, 1, activation='sigmoid')(conv9)


model1 = Model(inputs=inputs, outputs=conv10)
h=1
model1.summary()

In [ ]:
from skimage.metrics import peak_signal_noise_ratio, structural_similarity

def train(model, train_sharp, train_blur, val_sharp, val_blur, batch_size, epochs):
    opt = Adam(lr=1e-4)
    model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
    checkpoint = ModelCheckpoint('model1.h5', monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    callbacks_list = [checkpoint]
    model.fit(train_blur, train_sharp, batch_size=batch_size, epochs=epochs, validation_data=(val_blur, val_sharp), callbacks=callbacks_list)
    model.save("/content/drive/MyDrive/idb/saved_model/Unet/unetkeras1.h5")

def validate(model, val_sharp, val_blur):
    opt = Adam(lr=1e-4)
    model.compile(loss='mse', optimizer=opt, metrics=['accuracy'])
    model.load_weights('model1.h5')
    loss, acc = model1.evaluate(val_blur, val_sharp)
    print('Validation Loss:', loss)
    print('Validation Accuracy:', acc)

    pred_sharp = model.predict(val_blur)
    psnr = peak_signal_noise_ratio(val_sharp, pred_sharp)
    ssim = structural_similarity(val_sharp, pred_sharp, multichannel=True)
    print('PSNR:', psnr)
    print('SSIM:', ssim)
    return loss, acc

In [ ]:
train(model1, train_sharp, train_blur, val_sharp, val_blur, batch_size=32, epochs=15)
validate(model1, val_sharp, val_blur)